# Trying Out Numba

In [1]:
%pylab ipympl
%load_ext autoreload
%autoreload 2
import numba
from numba import jit
from numba import njit
import dose_calculator_numba as dcn

Populating the interactive namespace from numpy and matplotlib


In [15]:
%%time
n = 100000000

def add_a(n):
    a = 0
    b = 0
    for i in range(n):
        a += n
    for i in range(n):
        b += n
    return(a,b)


CPU times: user 17 µs, sys: 4 µs, total: 21 µs
Wall time: 36.7 µs


In [16]:
%%time
add_a(n)

CPU times: user 14.9 s, sys: 6.15 ms, total: 14.9 s
Wall time: 14.9 s


(10000000000000000, 10000000000000000)

In [17]:
%%time
@jit(nopython=True)
def add_a_fast(n):
    a = 0
    b = 0
    for i in range(n):
        a += n
    for i in range(n):
        b += n
    return(a,b)

CPU times: user 295 µs, sys: 36 µs, total: 331 µs
Wall time: 785 µs


In [25]:
%%time
add_a_fast(n)

CPU times: user 20 µs, sys: 2 µs, total: 22 µs
Wall time: 37.2 µs


(10000000000000000, 10000000000000000)

In [5]:
%%time 

# for a CT array of (Nx-1,Ny-1,Nz-1) voxels
Nx = 51
Ny = 51
Nz = 51

# distances between the x,y,z planes (also the lengths of the sides of the voxels) in cm
dx = 0.1
dy = 0.1
dz = 0.1

# initial and final coordinates of the beam
x1,x2 = (0,0)
y1,y2 = (0,0)
z1,z2 = (2.5,-2.5)

# adjustment from center 
adjust = 0.025

# initial plane coordinates
xplane1 = -2.5
yplane1 = -2.5
zplane1 = -2.5

# beam info and filename
beam_energy = 0.120 # in MeV
fluence_0 = 3.183098862 * 10**10 # photon/cm^2
# fluence_0 = 2.53 * 10**8 # photon/cm^2
# fluence_0 = 9.93 * 10**8 # photon/cm^2
# filename = 'energy_absorption_coeff.txt'

# angular (in radians) and positional spread (in cm)
angle_spread_x = 0
angle_spread_y = 0
angle_spread_z = 0

pos_spread_x = 0.01
pos_spread_y = 0.01
pos_spread_z = 0

angle_spread = (angle_spread_x,angle_spread_y,angle_spread_z)
position_spread = (pos_spread_x,pos_spread_y,pos_spread_z)

# file names for energy absorption coefficients
filenames = ['energy_absorption_coeffs.txt','energy_absorption_coeffs_lung.txt','energy_absorption_coeffs_cort_bone.txt']

# kernel info
kernelname_water = '../Topas/Kernels/WaterKernel13.csv'
kernelname_lung = '../Topas/Kernels/LungKernel1.csv'
kernelname_bone = '../Topas/Kernels/BoneKernel3.csv'
kernelnames = [kernelname_water,kernelname_lung,kernelname_bone]
# kernelname = '../Topas/RealKernel1.csv'
kernel_size = (4,4,4) # cm 

# effective distance from center of kernel 
eff_dist = (2,2,2) # cm

# making materials array 
materials = []
for i in range(Nx-1):
    materials.append([])
    for j in range(Ny-1):
        materials[i].append([])
        for k in range(Nz-1):
            materials[i][j].append('w')

# number of cores to use
num_cores = 16

# Siddon((Nx,Ny,Nz),(dx,dy,dz),((x1,x2),(y1,y2),(z1,z2)),(xplane1,yplane1,zplane1),plot=True)
# terma = sd.TERMA((Nx,Ny,Nz),(dx,dy,dz),((x1,x2),(y1,y2),(z1,z2)),(xplane1,yplane1,zplane1),[beam_energy],[fluence_0],mu)

dose = dcn.Dose_Calculator((Nx,Ny,Nz),(dx,dy,dz),[((x1+adjust,x2+adjust),(y1+adjust,y2+adjust),(z1,z2)),((x1-adjust,x2-adjust),(y1+adjust,y2+adjust),(z1,z2)),((x1+adjust,x2+adjust),(y1-adjust,y2-adjust),(z1,z2)),((x1-adjust,x2-adjust),(y1-adjust,y2-adjust),(z1,z2))],(xplane1,yplane1,zplane1),[beam_energy],[fluence_0],angle_spread,position_spread,filenames,kernelnames,kernel_size,eff_dist,materials,num_cores)


Calling Superposition
Kernel Interpolated


/home/user/NSERC-Summer-2022/Python/dose_calculator_numba.py:327: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Superimpose" failed type inference due to: No implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(list(array(float64, 2d, A))<iv=None>, int64, list(array(float64, 1d, A))<iv=None>)
 
There are 16 candidate implementations:
      - Of which 14 did not match due to:
      Overload of function 'setitem': File: <numerous>: Line N/A.
        With argument(s): '(list(array(float64, 2d, A))<iv=None>, int64, list(array(float64, 1d, A))<iv=None>)':
       No match.
      - Of which 2 did not match due to:
      Overload in function 'SetItemSequence.generic': File: numba/core/typing/collections.py: Line 56.
        With argument(s): '(list(array(float64, 2d, A))<iv=None>, int64, list(array(float64, 1d, A))<iv=None>)':
       Rejected as the implementation raised a specific error:
   

NameError: global name 'to_list' is not defined